In [2]:
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from sklearn.model_selection import train_test_split


In [3]:

from google.colab import files
uploaded = files.upload()


Saving GAN_Data_1.csv to GAN_Data_1.csv


In [4]:

# Load data
data = pd.read_csv("GAN_Data_1.csv")

In [5]:


# Split data into inputs and output
inputs = data.iloc[:, :4]
output = data.iloc[:, 4:]

In [6]:
# Split data into training and validation sets using 20/80 rule
train_data, val_data = train_test_split(data, test_size=0.2)

In [7]:

# Reset the index of the train_data dataframe
train_data = train_data.reset_index(drop=True)

In [8]:
# Normalize the inputs and output
inputs = (inputs - np.mean(inputs, axis=0)) / np.std(inputs, axis=0)
output = (output - np.mean(output, axis=0)) / np.std(output, axis=0)

In [9]:

# Set hyperparameters
latent_dim = 100
batch_size = 6
epochs = 500

In [10]:
# define the generator
#generator = Sequential()
#generator.add(Dense(10, input_dim=latent_dim, activation='relu'))
#generator.add(Dense(5, activation='tanh'))
#generator.add(Dense(5, activation='linear'))

# Define the generator
generator = Sequential()
generator.add(Dense(10, input_dim=latent_dim, activation='relu'))
generator.add(Dense(1, activation='linear'))

In [11]:
# define the discriminator
#discriminator = Sequential()
#discriminator.add(Dense(10, input_dim=5, activation='relu'))
#discriminator.add(Dense(5, activation='tanh'))
#discriminator.add(Dense(1, activation='sigmoid'))



# Define the discriminator
discriminator = Sequential()
discriminator.add(Dense(10, input_dim=5, activation='relu'))
discriminator.add(Dense(1, activation='sigmoid'))


In [12]:

# Compile discriminator model
discriminator.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [13]:

# Define the GAN model
gan_input = Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = Model(gan_input, gan_output)

ValueError: ignored

In [ ]:


# Compile GAN model
gan.compile(loss="binary_crossentropy", optimizer="adam")

In [ ]:


# Train the GAN model
for epoch in range(epochs):
    # Generate fake data
    noise = np.random.normal(0, 1, size=(batch_size, latent_dim))
    fake_output = generator.predict(noise)



In [ ]:
# Select a random batch of real data
idx = np.random.randint(0, train_data.shape[0], batch_size)
real_output = train_data.iloc[idx, -1].values.reshape(-1, 1)

In [ ]:
# Train the discriminator on real and fake data
discriminator_loss_real = discriminator.train_on_batch(real_output, np.ones((batch_size, 1)))
discriminator_loss_fake = discriminator.train_on_batch(fake_output, np.zeros((batch_size, 1)))
discriminator_loss = 0.5 * np.add(discriminator_loss_real, discriminator_loss_fake)


In [ ]:
 # Train the generator
noise = np.random.normal(0, 1, size=(batch_size, latent_dim))
gan_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))

   


In [ ]:
# Print losses and accuracy
    if epoch % 100 == 0:
        print(f"Epoch: {epoch+1}, Generator Loss: {generator_loss}, Discriminator Loss: {discriminator_loss[0]}, Discriminator Accuracy: {discriminator_loss[1]}")

In [ ]:

# Generate 25000 fake data using the generator (I need a sample of 25000)
noise = np.random.normal(0, 1, size=(25000, latent_dim))
fake_output = generator.predict(noise)

In [ ]:


# Rescale the output to original scale
fake_output = fake_output * np.std(output, axis=0) + np.mean(output, axis=0)



In [ ]:
# Save the fake data to a CSV file
fake_data = pd.DataFrame(data=fake_output, columns=['output'])
fake_data.to_csv('GAN_Fake_Data.csv', index=False)